<a href="https://colab.research.google.com/github/Carllllllll/Proposal/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
os.getcwd()

import pandas as pd

In [ ]:
df1 = pd.read_excel(os.getcwd() + '/drive/MyDrive/Proposal/data/propo1.xlsx', index_col=None, header=1)
df2 = pd.read_excel(os.getcwd() + '/drive/MyDrive/Proposal/data/propo2.xlsx', index_col=None, header=1)
df3 = pd.read_excel(os.getcwd() + '/drive/MyDrive/Proposal/data/propo3.xlsx', index_col=None, header=1)
df4 = pd.read_excel(os.getcwd() + '/drive/MyDrive/Proposal/data/propo4.xlsx', index_col=None, header=1)
df5 = pd.read_excel(os.getcwd() + '/drive/MyDrive/Proposal/data/propo5.xlsx', index_col=None, header=1)
df6 = pd.read_excel(os.getcwd() + '/drive/MyDrive/Proposal/data/propo6.xlsx', index_col=None, header=1)
df7 = pd.read_excel(os.getcwd() + '/drive/MyDrive/Proposal/data/propo7.xlsx', index_col=None, header=1)

In [ ]:
# 형태소 분석기 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 2.05 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 6.7 MB/s 
     |████████████████████████████████| 453 kB 39.1 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-06-30 06:03:33--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22e9:9f55, 2406:da00:ff00::22cd:e0db, ...
Connecting to bitbucket.org (bi

In [ ]:
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

In [ ]:
# print(df1.head())

df1 = df1.loc[df1['제안내용'] != '제안내용이 많아 하이미에서 확인요망']
df1 = df1.loc[df1['제안내용'] != '']

df2 = df2.loc[df2['제안내용'] != '제안내용이 많아 하이미에서 확인요망']
df2 = df2.loc[df2['제안내용'] != '']

df3 = df3.loc[df3['제안내용'] != '제안내용이 많아 하이미에서 확인요망']
df3 = df3.loc[df3['제안내용'] != '']

df4 = df4.loc[df4['제안내용'] != '제안내용이 많아 하이미에서 확인요망']
df4 = df4.loc[df4['제안내용'] != '']

df5 = df5.loc[df5['제안내용'] != '제안내용이 많아 하이미에서 확인요망']
df5 = df5.loc[df5['제안내용'] != '']

df6 = df6.loc[df6['제안내용'] != '제안내용이 많아 하이미에서 확인요망']
df6 = df6.loc[df6['제안내용'] != '']

df7 = df7.loc[df7['제안내용'] != '제안내용이 많아 하이미에서 확인요망']
df7 = df7.loc[df7['제안내용'] != '']

In [ ]:
df1 = df1[['제목', '제안내용']]
df2 = df2[['제목', '제안내용']]
df3 = df3[['제목', '제안내용']]
df4 = df4[['제목', '제안내용']]
df5 = df5[['제목', '제안내용']]
df6 = df6[['제목', '제안내용']]
df7 = df7[['제목', '제안내용']]

In [ ]:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7], ignore_index=True)

In [ ]:
df.head

<bound method NDFrame.head of                                   제목  \
0               갱신 알림톡 발송 화면 조회구분 개선   
1      완전판매시스템 현소속지점 기준으로 조회가능하도록 개선   
2                   철회 환급시 구비 서류 증빙    
3             이관고객관리 활동프로세스 관리대상변경요청   
4                         이관 메모기능 추가   
...                              ...   
20055       자동차 첨부사진 서식명 변경 가능하도록 요청   
20056                S포탈 집금책임화면 수정요청   
20057                    동일특약 상품명 상이   
20058         은행 거래내역 수신용 PC의 서버화 요청   
20059          가상계좌입금조회 담당자조회가능하게 요청   

                                                    제안내용  
0      1. 제안배경      * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성  ...  
1      1. 제안배경      * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성  ...  
2      1. 제안배경      * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성  ...  
3      1. 제안배경    이관고객관리 활동프로세스가 시행되면서 이관받은 고객은 터치활동하...  
4      1. 제안배경      * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성  ...  
...                                                  ...  
20055  1. 제안배경     * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성   ...  
20056  1. 제안배경  현

In [ ]:
list(df.columns)

['제목', '제안내용']

In [ ]:
df = df[['제목', '제안내용']]

In [ ]:
df = df.loc[df['제안내용'] != '제안내용이 많아 하이미에서 확인요망']

In [ ]:
df.dropna

<bound method DataFrame.dropna of                                   제목  \
0               갱신 알림톡 발송 화면 조회구분 개선   
1      완전판매시스템 현소속지점 기준으로 조회가능하도록 개선   
2                   철회 환급시 구비 서류 증빙    
3             이관고객관리 활동프로세스 관리대상변경요청   
4                         이관 메모기능 추가   
...                              ...   
20386       자동차 첨부사진 서식명 변경 가능하도록 요청   
20387                S포탈 집금책임화면 수정요청   
20388                    동일특약 상품명 상이   
20389         은행 거래내역 수신용 PC의 서버화 요청   
20390          가상계좌입금조회 담당자조회가능하게 요청   

                                                    제안내용  
0      1. 제안배경      * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성  ...  
1      1. 제안배경      * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성  ...  
2      1. 제안배경      * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성  ...  
3      1. 제안배경    이관고객관리 활동프로세스가 시행되면서 이관받은 고객은 터치활동하...  
4      1. 제안배경      * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성  ...  
...                                                  ...  
20386  1. 제안배경     * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성   ...  
20387  1. 제안배

In [ ]:
# mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic")
mecab = Mecab()
tagged_corpus_list = []

for index, row in tqdm(df.iterrows(), total=len(df)):
    
    text = row['제안내용']
    tag = row['제목']
    if pd.isna(text): continue
    if pd.isna(tag): continue
    tagged_corpus_list.append(TaggedDocument(tags=[tag], words=mecab.morphs(text)))

print('문서의 수 :', len(tagged_corpus_list))

100%|██████████| 20060/20060 [00:23<00:00, 846.08it/s]

문서의 수 : 18236


In [ ]:
tagged_corpus_list[0]

TaggedDocument(words=['1', '.', '제안', '배경', '*', '제안', '사항', '의', '현황', '/', '프로세스', '또는', '개선', '대상', '/', '문제점', '작성', '갱신', '도래', '월도', ',', '연령', '별', '조회', '안', '됨', '2', '.', '개선', '안', '(', '제안', '내용', ')', '*', '제안', '내용', '의', '개선', '방안', '또는', '개선', '가능', '방향', '에', '대해', '작성', '갱신', '도래', '월도', ',', '연령', '조회', '구분', '추가', '요청', '.', '갱신', '앞둔', '계약', ',', '고연', '령자', '위주', '로', '터치', '가능', '.', '3', '.', '예상', '효과', '※', '성과', '우수', '제안', '선정', '의', '주요', '기준', '이', '므로', '상세', '하', '게', '기술', '바랍니다', '.', '(', '1', ')', '재무', '성과', '(', '年', '기준', '/', '천', '원', '단위', ')', '(', '2', ')', '업무', '효율', '화', '(', '年', '기준', ')', '전환', '실', '손', '관리', '효율', '적', '(', '3', ')', '기타', '효과', '※', '예시', '활용', '목적', '으로', '신분', '확인', '서류', '등', '의', '첨', '부시', '개인', '정보', '가', '노출', '되', '지', '않', '도록', '각별히', '주의', '바랍니다', '.'], tags=['갱신 알림톡 발송 화면 조회구분 개선'])

In [ ]:
from gensim.models import doc2vec

In [ ]:
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)
# print(f"Tag Size: {len(model.docvecs.doctags.keys())}", end=' / ')

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

# 모델 저장
model.save('/content/drive/MyDrive/Proposal/dart.doc2vec')

In [ ]:
import os
os.getcwd()
os.chdir('/content/drive/MyDrive/Proposal/')

In [ ]:
df[40:43]

,제목,제안내용
40,간편심사평상품 동시가입 건에 대한 계약 매칭,1. 제안배경 '간편심사형 상품' 과 '일반심사형 상품' 동시설계시 적정성...
42,하이카서비스(중형 화물) 확대요청,1. 제안배경 * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성 ...
43,장기신계약 입력시 신용카드 등록 동시 진행 가능하도록 개선요청,1. 제안배경 신계약 청약시 자동이체 미설정계약에 대한 자동이체잋 요청시 ...


In [ ]:
df.at[1, '제목']

'완전판매시스템 현소속지점 기준으로 조회가능하도록 개선'

In [ ]:
# df.iat[i, 1]
i = 5
str = df.at[i, '제목']
print(str)

점포장/직원 sms 화면 개선요청


In [ ]:
similar_doc = model.docvecs.most_similar(str)
print(similar_doc)

[('영업포탈 입금표화면 개선요청', 0.4098973572254181), ('2020년3월이후계약 단기운전자 배서 휴대폰서명 안됨', 0.40506404638290405), ('점포별 하이플래너 명단 채널구분 사용인으로 조회시 연락처 조회가능요청', 0.4034386873245239), ('공제내역조회 개선바랍니다.', 0.4019550085067749), ('지급보증서 발송 기준 개선', 0.39920681715011597), ('완전판매 조치대상 화면에서  계약조회화면 연결,  e모니터링 문자 발송 바로 되었으면 합니다. ', 0.3880622982978821), ('갱신 알림톡 발송 화면 조회구분 개선', 0.38382017612457275), ('암진단코드 C77~C80 진단시 약관상 유의사항부분 상품설명서상에  자세히 기재  요청 및 플래너 재교육요청', 0.382213830947876), ('하이포탈 플래너조회 권한요청', 0.38009142875671387), ('설계명세조회시 지점코드로 산하소속팀(SA팀)까지 조회가능 요청', 0.37833964824676514)]


In [ ]:
str2 = similar_doc[1][0]

In [ ]:
df.loc[df['제목'] == str].iat[0,1]

'1. 제안배경      * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성     인사발령으로 소속지점이 바뀐 경우 완전판매시스템에서 발령 이후 조치대상 계약건만 조회되고  이전 조치대상의 경우 조회되지않습니다.   (ex. 12월 인사발령의 경우 11월 조치대상계약 조회가 안됨)     이 경우 기존(이전)총무에게 요청하는 등 업무처리가 번거롭습니다.        2. 개선안(제안내용)     * 제안내용의 개선방안 또는 개선가능 방향에 대해 작성     현점포 기준으로 완전판매 조치대상 계약 전건을 볼 수 있도록 전산 개선 바랍니다.         3. 예상효과     효율적인 완판 관리'

In [ ]:
df.loc[df['제목'] == str2].iat[0,1]

'1. 제안배경      * 2020년 3월 이후개시계약 단기운전자 특약 휴대폰서명 안됨        2. 개선안(제안내용)     * 3월 이전 계약과 동일하게 휴대폰직접서명 가능하도록 요청        3. 예상효과     * 플래너분들이 직접서명이 되지 않아 원스탑으로 업무처리가 되지 않는다고 불만을 호소함       기존처럼 원스탑으로 처리가 되면 불만을 저희 에게 호소하지 않으며 원만한 업무처리가 진행됨'

In [ ]:
similar_doc[1][0]

'지급보증서 발송 기준 개선'

In [ ]:
# 정확도 높은 건들 찾기

for i in range(1,len(df)):
    
    try:
        str = df.at[i, '제목']
        similar_doc = model.docvecs.most_similar(str)
    except:
        a = 3

    if similar_doc[1][1] > 0.7:
        print(i)
        print(str)
        print(similar_doc[1][0])

2250
조치대상 전산제안
조치대상 화면 제안
4804
조치대상 전산제안
조치대상 화면 제안
5891
완전판매확인서 발행시 완판콜센터번호 추가요청
지급보증서 발송 기준 개선
6118
조치대상 화면 보완
조치대상 전산제안
8583
암진단코드 C77~C80 진단시 약관상 유의사항부분 상품설명서상에  자세히 기재  요청 및 플래너 재교육요청
간병비 지급대상 전산로직 개선
8839
조치대상 화면 제안
조치대상 전산제안
8840
보험사기 할증보험료 환급 업무 절차 개선
조치대상 전산제안
8841
변호사 소송위임시 임시저장 기능 
조치대상 전산제안
11052
의료자문 동의서 하이손사 게시판에 등록
TM자동차보험(신규+타사유입차량) 블랙박스 사진 미첨부 할인요청
11053
추상장해 기준 수립
암진단코드 C77~C80 진단시 약관상 유의사항부분 상품설명서상에  자세히 기재  요청 및 플래너 재교육요청
11596
지급보증서 발송 기준 개선
자녀성과수수료 입력 관련
12165
증권증권 재발행 확대
장기 증권 재발행 요청
14353
소급부활지연입력 개선요청
소급부활 지역입력 개선요청
14443
TM자동차보험(신규+타사유입차량) 블랙박스 사진 미첨부 할인요청
간병비 지급대상 전산로직 개선
14696
소급계약 지연입력 개선요청
소급부활 지역입력 개선요청
14698
소급부활 지역입력 개선요청
소급계약 지연입력 개선요청
15857
간병비 지급대상 전산로직 개선
추상장해 기준 수립
16207
자녀성과수수료 입력 관련
TM자동차보험(신규+타사유입차량) 블랙박스 사진 미첨부 할인요청
16444
장기 증권 재발행 요청
증권증권 재발행 확대
18460
의료기관별 합의참고정보 활용 고도화 방안
피해자와 합의후 건보공단구상금처리문제
18709
피해자와 합의후 건보공단구상금처리문제
암진단코드 C77~C80 진단시 약관상 유의사항부분 상품설명서상에  자세히 기재  요청 및 플래너 재교육요청


In [ ]:
df.at[2250, '제안내용']

'1. 제안배경         * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성           매일 조치대상에서 완전판매조치 안내 및 처리를 하고있습니다.     오전/오후 나누어 보고있는데     종종 처리할수없는 (해외, 야간일을하여 오전모니터링 불가등) 건들을 플래너에게 전달받았음에도 불구하고     다음날 또 처리가 안되어 이중으로 연락을 하게됩니다.     또한 후임총무와 함께 업무처리를 하게되면 한 플래너에게 이중연락등 중복되게됩니다.     2. 개선안(제안내용)        * 제안내용의 개선방안 또는 개선가능 방향에 대해 작성            하단에 메모기능을 추가하여     메모가 되어있는부분은 처리전 한번더  볼수있도록 제안합니다.           3. 예상효과        ※ 성과우수제안 선정의 주요기준이므로 상세하게 기술바랍니다.           (3) 기타 효과           시간감소, 효율적 업무처리 가능        ※  예시 활용목적으로 신분확인 서류 등의 첨부시 개인정보가 노출되지 않도록 각별히 주의바랍니다.'

In [ ]:
str = df.at[2250, '제목']
similar_doc = model.docvecs.most_similar(str)

In [ ]:
similar_doc

[('조치대상 화면 보완', 0.847792387008667),
 ('조치대상 화면 제안', 0.8416188955307007),
 ('대리점 조치대상화면 수정요청합니다.', 0.43335217237472534),
 ('콜적부 조치기한 알람 뜨도록 개선요청', 0.41208720207214355),
 ('플래너 본인 계약 시 모집자 서명 안해도 확정가능하도록 제안합니다. ', 0.39692389965057373),
 ('완전판매 모니터링 최종차수 조치내용 관련', 0.39235469698905945),
 ('플래너 본인계약 모집자서명 생략', 0.38509178161621094),
 ('완전판매 모니터링 대상 리스트 제공필요', 0.375488817691803),
 ('하이포탈에서 어린이보험 설계건 조회 요청', 0.3726857304573059),
 ('장기 계속보험료 입금 취소건 지점 전결 처리', 0.3718041181564331)]

In [ ]:
df.loc[df['제목'] == '조치대상 화면 보완'].iat[0,1]

'1. 제안배경      * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성     매일 조치대상에서 완전판매조치 안내 및 처리를 하고있습니다.  오전/오후 나누어 보고있는데  종종 처리할수없는 (해외, 야간일을하여 오전모니터링 불가등) 건들을 플래너에게 전달받았음에도 불구하고  다음날 또 처리가 안되어 이중으로 연락을 하게됩니다.     2. 개선안(제안내용)     * 제안내용의 개선방안 또는 개선가능 방향에 대해 작성      하단에 메모기능을 추가하여  메모가 되어있는부분은 처리전 한번더  볼수있도록 제안합니다.     3. 예상효과     ※ 성과우수제안 선정의 주요기준이므로 상세하게 기술바랍니다.        (3) 기타 효과     시간감소, 효율적 업무처리 가능     ※  예시 활용목적으로 신분확인 서류 등의 첨부시 개인정보가 노출되지 않도록 각별히 주의바랍니다.'

In [ ]:
df.loc[df['제목'] == '조치대상 화면 제안'].iat[0,1]

'1. 제안배경      * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성     매일 조치대상에서 완전판매조치 안내 및 처리를 하고있습니다.  오전/오후 나누어 보고있는데  종종 처리할수없는 (해외, 야간일을하여 오전모니터링 불가등) 건들을 플래너에게 전달받았음에도 불구하고  다음날 또 처리가 안되어 이중으로 연락을 하게됩니다.     2. 개선안(제안내용)     * 제안내용의 개선방안 또는 개선가능 방향에 대해 작성      하단에 메모기능을 추가하여  메모가 되어있는부분은 처리전 한번더  볼수있도록 제안합니다.     3. 예상효과     ※ 성과우수제안 선정의 주요기준이므로 상세하게 기술바랍니다.        (3) 기타 효과     시간감소, 효율적 업무처리 가능     ※  예시 활용목적으로 신분확인 서류 등의 첨부시 개인정보가 노출되지 않도록 각별히 주의바랍니다.'